In [78]:
import warnings
import itertools
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from pmdarima.arima import ARIMA

In [2]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

## Load and prepare data:

In [3]:
df = pd.read_csv('data/data.csv', delimiter=';')
df.columns = ["date", "year", "month", "week", "day", "cnt", "average_sum"]
df["average_sum"] = df["average_sum"].apply(lambda x: float(x.replace(",", ".")))
df = df.set_index("date")
df = df.sort_values(["year", "month", "week", "day"])

In [81]:
values = df['cnt'].values
ts_log = np.log(values)

## Split dataset into train and test sets:

In [82]:
train, test = train_test_split(ts_log, shuffle=False)

## Train ARIMA model:

In [107]:
model = ARIMA(order=(1, 0, 2), seasonal_order=(0, 0, 0, 12))
model.fit(train)

/home/slyfest/anaconda3/lib/python3.7/site-packages/statsmodels/base/model.py:508: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  "Check mle_retvals", ConvergenceWarning)


ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(1, 0, 2),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 0, 0, 12), solver='lbfgs', start_params=None,
   suppress_warnings=False, transparams=True, trend=None,
   with_intercept=True)

## Revert data into previous form and calculate MAPE:

In [108]:
preds = model.predict(183)
preds = np.exp(preds)

test_y = np.exp(test)

print(mean_absolute_percentage_error(test_y, preds))

24.398749640542086


In [109]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [112]:
future = model.predict(206)
future = np.exp(future[23:])